# RAG with Mistral AI, Azure AI Search and Azure AI Studio

## Overview

This notebook demonstrates how to integrate Mistral Embeddings with Azure AI Search as a vector store, and use the results to ground responses in the Mistral Chat Completion Model.

## Prerequisites

- Mistral AI API Key OR Azure AI Studio Deployed Mistral Chat Completion Model and Azure AI Studio API Key
- Azure AI Search service
- Python 3.x environment with necessary libraries installed

## Steps

1. Install required packages
2. Load data and generate Mistral embeddings
3. Index embeddings in Azure AI Search
4. Perform search using Azure AI Search
5. Ground search results in Mistral Chat Completion Model


## Install Required Packages



In [ ]:
# Install Required Packages
!pip install azure-search-documents==11.6.0b4
!pip install azure-identity==1.16.0 datasets==2.19.1 mistralai==0.2.0

## Load Data and Generate Mistral Embeddings

In [11]:
from datasets import load_dataset

data = load_dataset(
    "jamescalam/ai-arxiv2-semantic-chunks",
    split="train[:10000]"
)
data

Dataset({
    features: ['id', 'title', 'content', 'prechunk_id', 'postchunk_id', 'arxiv_id', 'references'],
    num_rows: 10000
})

We have 10K chunks, where each chunk is roughly the length of 1-2 paragraphs in length. Here is an example of a single record:

In [12]:
data[0]

{'id': '2401.04088#0',
 'title': 'Mixtral of Experts',
 'content': '4 2 0 2 n a J 8 ] G L . s c [ 1 v 8 8 0 4 0 . 1 0 4 2 : v i X r a # Mixtral of Experts Albert Q. Jiang, Alexandre Sablayrolles, Antoine Roux, Arthur Mensch, Blanche Savary, Chris Bamford, Devendra Singh Chaplot, Diego de las Casas, Emma Bou Hanna, Florian Bressand, Gianna Lengyel, Guillaume Bour, Guillaume Lample, LÃ©lio Renard Lavaud, Lucile Saulnier, Marie-Anne Lachaux, Pierre Stock, Sandeep Subramanian, Sophia Yang, Szymon Antoniak, Teven Le Scao, ThÃ©ophile Gervet, Thibaut Lavril, Thomas Wang, TimothÃ©e Lacroix, William El Sayed Abstract We introduce Mixtral 8x7B, a Sparse Mixture of Experts (SMoE) language model. Mixtral has the same architecture as Mistral 7B, with the difference that each layer is composed of 8 feedforward blocks (i.e. experts). For every token, at each layer, a router network selects two experts to process the current state and combine their outputs. Even though each token only sees two experts

Format the data into the format we need, this will contain `id`, `title`, `content` (which we will embed), and `arxiv_id`.

In [13]:
data = data.remove_columns(["prechunk_id", "postchunk_id", "references"])
data

Dataset({
    features: ['id', 'title', 'content', 'arxiv_id'],
    num_rows: 10000
})

We need to define an embedding model to create our embedding vectors for retrieval, for that we will be using Mistral AI's `mistral-embed`. There is some cost associated with this model, so be aware of that (costs for running this notebook are <$1).


In [14]:
import os
from mistralai.client import MistralClient
import getpass  # console.mistral.ai/api-keys/

mistral_api_key = os.getenv("MISTRAL_API_KEY") or getpass.getpass("Enter your Mistral API key: ")

mistral = MistralClient(api_key=mistral_api_key)


In [15]:
embed_model = "mistral-embed"

embeds = mistral.embeddings(
    model=embed_model, input=["this is a test"]
)

We can view the dimensionality of our returned embeddings, which we'll need soon when initializing our vector index:

In [16]:
dims = len(embeds.data[0].embedding)
dims

1024

## Index Embeddings into Azure AI Search

Now we create our vector DB to store our vectors. For this, we need to set up an [Azure AI Search service](https://portal.azure.com/#create/Microsoft.Search).

There are two ways to authenticate to Azure AI Search:

1. **Service Key**: The service key can be found in the "Settings -> Keys" section in the left navbar of the Azure portal dashboard. Make sure to select the ADMIN key.
2. **Managed Identity**: Using Microsoft Entra ID (f.k.a. Azure Active Directory) is a more secure and recommended way to authenticate. You can follow the instructions in the [official Microsoft documentation](https://learn.microsoft.com/azure/search/search-security-rbac) to set up Managed Identity.

For more detailed instructions on creating an Azure AI Search service, please refer to the [official Microsoft documentation](https://learn.microsoft.com/azure/search/search-create-service-portal).


### Authenticate into Azure AI Search

In [20]:
from getpass import getpass
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

# Configuration variable
USE_AAD_FOR_SEARCH = True

SEARCH_SERVICE_ENDPOINT = os.getenv("SEARCH_SERVICE_ENDPOINT") or getpass("Enter your Azure AI Search Service Endpoint: ")

def authenticate_azure_search(use_aad_for_search=False):
    if use_aad_for_search:
        print("Using AAD for authentication.")
        credential = DefaultAzureCredential()
    else:
        print("Using API keys for authentication.")
        api_key = os.getenv("SEARCH_SERVICE_API_KEY") or getpass("Enter your Azure AI Search Service API Key: ")
        if api_key is None:
            raise ValueError("API key must be provided if not using AAD for authentication.")
        credential = AzureKeyCredential(api_key)
    return credential

azure_search_credential = authenticate_azure_search(
    use_aad_for_search=USE_AAD_FOR_SEARCH
)


Using AAD for authentication.


### Create a vector index

In [21]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex,
)

DIMENSIONS = 1024
HNSW_PARAMETERS = {"m": 4, "metric": "cosine", "ef_construction": 400, "ef_search": 500}
INDEX_NAME = "ai-arxiv2-semantic-chunks"  # replace with your actual index name


def create_index(endpoint, credential):
    # Create a search index
    index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
    fields = [
        SimpleField(
            name="id",
            type=SearchFieldDataType.String,
            key=True,
            sortable=False,
            filterable=True,
            facetable=False,
        ),
        SearchableField(name="title", type=SearchFieldDataType.String),
        SearchableField(name="content", type=SearchFieldDataType.String),
        SearchableField(
            name="arxiv_id", type=SearchFieldDataType.String, filterable=True
        ),
        SearchField(
            name="embedding",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=DIMENSIONS,
            vector_search_profile_name="myHnswProfile",
            hidden=False,  # Set to true if you want to retrieve the embeddings in the search results
        ),
    ]

    # Configure the vector search configuration
    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(name="myHnsw", parameters=HNSW_PARAMETERS)
        ],
        profiles=[
            VectorSearchProfile(
                name="myHnswProfile",
                algorithm_configuration_name="myHnsw",
            )
        ],
    )

    semantic_config = SemanticConfiguration(
        name="my-semantic-config",
        prioritized_fields=SemanticPrioritizedFields(
            title_field=SemanticField(field_name="title"),
            keywords_fields=[SemanticField(field_name="arxiv_id")],
            content_fields=[SemanticField(field_name="content")],
        ),
    )

    # Create the semantic settings with the configuration
    semantic_search = SemanticSearch(configurations=[semantic_config])

    # Create the search index with the semantic settings
    index = SearchIndex(
        name=INDEX_NAME,
        fields=fields,
        vector_search=vector_search,
        semantic_search=semantic_search,
    )
    result = index_client.create_or_update_index(index)
    print(f" {result.name} created")
    
create_index(SEARCH_SERVICE_ENDPOINT, azure_search_credential)

 ai-arxiv2-semantic-chunks created


### Estimate Cost for Embedding Generation

As per the information from [Lunary.ai's Mistral Tokenizer](https://lunary.ai/mistral-tokenizer), one token is approximately equivalent to five characters of text. 

According to [Mistral's Pricing](https://mistral.ai/technology/#pricing), the cost for using `mistral-embed` is $0.1 per 1M tokens for both inputs and outputs.

In the following code block, we will calculate the estimated cost for generating embeddings based on the size of our dataset and these pricing details.

In [22]:
def estimate_cost(data, cost_per_million_tokens=0.1):
    total_characters = sum(len(entry['content']) for entry in data)
    total_tokens = total_characters / 5  # 1 token is approximately 5 characters
    total_tokens_in_millions = total_tokens / 1_000_000
    total_cost = total_tokens_in_millions * cost_per_million_tokens
    return total_cost

estimated_cost = estimate_cost(data)
print(f"Estimated cost for generating embeddings: ${estimated_cost}")

Estimated cost for generating embeddings: $0.19047898000000002


### Transform Dataset for Azure AI Search Upload

In [31]:
# Function to transform your dataset into the format required by Azure AI Search
def transform_to_search_document(record):
    return {
        "id": record["id"],
        "arxiv_id": record["arxiv_id"],
        "title": record["title"],
        "content": record["content"]
    }
    
# Transform all documents in the dataset
transformed_documents = [transform_to_search_document(doc) for doc in data]

### Generate Embeddings

In [36]:
def generate_embeddings(documents, model, batch_size=20):
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        contents = [doc['content'] for doc in batch]
        embeds = mistral.embeddings(model=model, input=contents)
        for j, document in enumerate(batch):
            document['embedding'] = embeds.data[j].embedding
    return documents

embed_model = "mistral-embed"
generate_embeddings(transformed_documents, embed_model)

Azure AI Search doesn't allow certain unsafe keys so we'll base64 encode `id` here

In [38]:
import base64

def encode_key(key):
    return base64.urlsafe_b64encode(key.encode()).decode()

# Use the function to encode keys
for document in transformed_documents:
    document['id'] = encode_key(document['id'])

### Upload Documents

In [43]:
from azure.search.documents import SearchClient
from azure.search.documents import SearchIndexingBufferedSender


def upload_documents(index_name, endpoint, credential, documents):
    buffered_sender = SearchIndexingBufferedSender(endpoint=endpoint, index_name=index_name, credential=credential)

    for document in documents:
        buffered_sender.upload_documents(documents=[document])

    # Flush remaining documents and wait for completion
    buffered_sender.flush()

    print(f"Uploaded {len(documents)} documents in total")

# Use the function to upload documents
upload_documents(
    INDEX_NAME,
    SEARCH_SERVICE_ENDPOINT,
    azure_search_credential,
    transformed_documents,
)

Uploaded 10000 documents in total


## Perform a Vector Search

In [44]:
# Initialize the SearchClient
search_client = SearchClient(
    endpoint=SEARCH_SERVICE_ENDPOINT, 
    index_name=INDEX_NAME, 
    credential=azure_search_credential
)

In [45]:
from azure.search.documents.models import (
    VectorizedQuery
)

def generate_query_embedding(query):
    # Set the embedding model
    embed_model = "mistral-embed"

    # Generate embedding for the query
    embed = mistral.embeddings(model=embed_model, input=[query])
    return embed.data[0].embedding

# Pure Vector Search
query = "can you tell me about mistral LLM?"

vector_query = VectorizedQuery(
    vector=generate_query_embedding(query), k_nearest_neighbors=3, fields="embedding"
)

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    select=["id", "arxiv_id", "title", "content"],
)

for result in results:
    print(f"ID: {result['id']}")
    print(f"Arxiv ID: {result['arxiv_id']}")
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print("-" * 50)

ID: TWpRd01TNHdOREE0T0NNNQ==
Arxiv ID: 2401.04088
Title: Mixtral of Experts
Score: 0.7831388
Content: Mistral 78 % 2681 Mistral 78 3 3 s0 5 = A % 66 50 g 4 45 64 78 138 348708 78 138 348708 78 138 348 70B S66 Mixtral 8x7B 50 Mixtral 8x7B 5 = 564 340 g al Mistral 78 ee Mistral 78 3 5 Â§ 30 5 eo â = Mistral Â° 20 â e LlaMA2 78 (138 348 70B 7B (138 348 708 7B Â«13B 34B 708 Active Params Active Params Active Params Figure 3: Results on MMLU, commonsense reasoning, world knowledge and reading comprehension, math and code for Mistral (7B/8x7B) vs Llama 2 (7B/13B/70B). Mixtral largely outperforms Llama 2 70B on all benchmarks, except on reading comprehension benchmarks while using 5x lower active parameters. It is also vastly superior to Llama 2 70B on code and math. Detailed results for Mixtral, Mistral 7B and Llama 2 7B/13B/70B and Llama 1 34B2 are reported in Table 2. Figure 2 compares the performance of Mixtral with the Llama models in different categories. Mixtral surpasses Llama 2 70B a

## Perform a Hybrid Search

In [46]:
def generate_query_embedding(query):
    # Set the embedding model
    embed_model = "mistral-embed"

    # Generate embedding for the query
    embed = mistral.embeddings(model=embed_model, input=[query])
    return embed.data[0].embedding

# Pure Vector Search
query = "can you tell me about mistral LLM?"

vector_query = VectorizedQuery(
    vector=generate_query_embedding(query), k_nearest_neighbors=3, fields="embedding"
)

results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    select=["id", "arxiv_id", "title", "content"],
    top=3 # Limit the number of results
)

for result in results:
    print(f"ID: {result['id']}")
    print(f"Arxiv ID: {result['arxiv_id']}")
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print("-" * 50)

ID: TWpNeE1DNHdOamd5TlNNeA==
Arxiv ID: 2310.06825
Title: Mistral 7B
Score: 0.03201844170689583
Content: Mistral 7B outperforms the previous best 13B model (Llama 2, [26]) across all tested benchmarks, and surpasses the best 34B model (LLaMa 34B, [25]) in mathematics and code generation. Furthermore, Mistral 7B approaches the coding performance of Code-Llama 7B [20], without sacrificing performance on non-code related benchmarks. Mistral 7B leverages grouped-query attention (GQA) [1], and sliding window attention (SWA) [6, 3]. GQA significantly accelerates the inference speed, and also reduces the memory requirement during decoding, allowing for higher batch sizes hence higher throughput, a crucial factor for real-time applications. In addition, SWA is designed to handle longer sequences more effectively at a reduced computational cost, thereby alleviating a common limitation in LLMs. These attention mechanisms collectively contribute to the enhanced performance and efficiency of Mistra

## Ground retrieved results from Azure AI Search to Mistral-Large LLM

In [49]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

query = "can you tell me about mistral LLM?"

# Initialize the client
client = MistralClient(api_key=mistral_api_key)

# Convert the results to a string format
context = "\n---\n".join([f"ID: {result['id']}\nArxiv ID: {result['arxiv_id']}\nTitle: {result['title']}\nScore: {result['@search.score']}\nContent: {result['content']}" for result in results])

# Create the system message
system_message = (
    "You are a helpful assistant that answers questions about AI using the "
    "context provided below.\n\n"
    "CONTEXT:\n"
    + context
)

# Create the user message
user_message = "can you tell me about mistral LLM?"

# Create the messages parameter
messages = [
    ChatMessage(
        role="system",
        content=system_message
    ),
    ChatMessage(
        role="user",
        content=user_message
    )
]

# Generate the response
chat_response = client.chat(
    model="mistral-large-latest",
    messages=messages,
    max_tokens=50,
)

print(chat_response.choices[0].message.content)



Mistral AI is a company based in Paris, France, developing large language models (LLMs). Their primary product is a large language model that is designed to understand and generate human-like text based on the input it receives. This model has been


## Ground Results to Mistral-Large hosted in Azure AI Studio

In [58]:
# get API key from left navbar in Mistral console
azure_ai_studio_mistral_base_url = os.getenv("AZURE_AI_STUDIO_MISTRAL_BASE_URL") or getpass.getpass("Enter your Azure Mistral Deployed Endpoint Base URL: ") # Example: https://<endpoint-name>.<region>.inference.ai.azure.com"
azure_ai_studio_mistral_api_key = os.getenv("AZURE_AI_STUDIO_MISTRAL_API_KEY") or getpass.getpass("Enter your Azure Mistral API Key: ")

In [65]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

# Initialize the client
client = MistralClient(
    endpoint=azure_ai_studio_mistral_base_url, 
    api_key=azure_ai_studio_mistral_api_key
)

# Convert the results to a string format
context = "\n---\n".join([f"ID: {result['id']}\nArxiv ID: {result['arxiv_id']}\nTitle: {result['title']}\nScore: {result['@search.score']}\nContent: {result['content']}" for result in results])

# Create the system message
system_message = (
    "You are a helpful assistant that answers questions about AI using the "
    "context provided below.\n\n"
    "CONTEXT:\n"
    + context
)

# Create the user message
user_message = "can you tell me about mistral LLM?"

# Create the messages parameter
messages = [
    ChatMessage(
        role="system",
        content=system_message
    ),
    ChatMessage(
        role="user",
        content=user_message
    )
]

# Generate the response
chat_response = client.chat(
    model="azureai",
    messages=messages,
    max_tokens=50,
)

print(chat_response.choices[0].message.content)

Mistral AI is a cutting-edge company based in Paris, France, developing large language models (LLMs). Their mission is to build safe, robust, and beneficial AI systems that can understand and generate natural language text.

Mistral
